In [1]:
import os
import warnings
import keras
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import newaxis
from keras.layers.core import  Activation, Dropout
from keras.models import Model
from keras.layers import Input, Embedding,LSTM, Dense,TimeDistributed
from keras import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
def series_to_supervised(data, n_in, n_out, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(),list()
    for i in range(n_in, 0 ,-1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis =1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace = True)
    return agg

def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()

In [3]:
dataset_stock = read_csv('stock1.csv', header = 0, index_col=0)
values_stock = dataset_stock.values
encoder = LabelEncoder()
values_stock[:,0] = encoder.fit_transform(values_stock[:,0])
values_stock = values_stock.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_stock = scaler.fit_transform(values_stock)
reframed_stock = series_to_supervised(scaled_stock, 1, 1)
print(reframed_stock.head())

   var1(t-1)   var1(t)
1   0.157798  0.161468
2   0.161468  0.150459
3   0.150459  0.177982
4   0.177982  0.166972
5   0.166972  0.179817


In [4]:
dataset_gap = read_csv('gap1.csv', header = 0, index_col=0)
values_gap = dataset_gap.values
encoder = LabelEncoder()
values_gap[:,0] = encoder.fit_transform(values_gap[:,0])
values_gap = values_gap.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_gap = scaler.fit_transform(values_gap)
reframed_gap = series_to_supervised(scaled_gap, 1, 1)
print(reframed_gap.head())

   var1(t-1)   var1(t)
1   0.488038  0.435407
2   0.435407  0.617225
3   0.617225  0.401914
4   0.401914  0.559809
5   0.559809  0.397129


In [5]:
values_stock = reframed_stock.values
n_train_hours = 80
n_end = n_train_hours + 5
train_stock = values_stock[:n_train_hours, :]

test_stock = values_stock[n_train_hours: n_end :]

train_X_stock, train_y_stock = train_stock[:,:-1], train_stock[:, -1]

test_stock_X, test_stock_y = test_stock[:,:-1],test_stock[:,-1]

train_X_stock = train_X_stock.reshape((train_X_stock.shape[0], 1, train_X_stock.shape[1]))
test_stock_X = test_stock_X.reshape((test_stock_X.shape[0], 1, test_stock_X.shape[1]))
print(train_X_stock.shape, train_y_stock.shape, test_stock_X.shape, test_stock_y.shape)

(80, 1, 1) (80,) (5, 1, 1) (5,)


In [6]:
stock_input = Input(shape=(train_X_stock.shape[1], train_X_stock.shape[2]), dtype='float32', name='stock_input')
print (stock_input)

Tensor("stock_input:0", shape=(?, 1, 1), dtype=float32)


In [7]:
lstm_out_stock = LSTM(32)(stock_input)
lstm_out_stock = Dense(1)(lstm_out_stock)
print (lstm_out_stock)

Tensor("dense_1/BiasAdd:0", shape=(?, 1), dtype=float32)


In [8]:
values_gap = reframed_gap.values
train_gap = values_gap[:n_train_hours, :]

test_gap = values_gap[n_train_hours:n_end :]

train_X_gap, train_y_gap = train_gap[:,:-1], train_gap[:, -1]

test_gap_X, test_gap_y = test_gap[:,:-1],test_gap[:,-1]

train_X_gap = train_X_gap.reshape((train_X_gap.shape[0], 1, train_X_gap.shape[1]))
test_gap_X = test_gap_X.reshape((test_gap_X.shape[0], 1, test_gap_X.shape[1]))
print(train_X_gap.shape, train_y_gap.shape, test_gap_X.shape, test_gap_y.shape)

(80, 1, 1) (80,) (5, 1, 1) (5,)


In [9]:
gap_input = Input(shape=(train_X_gap.shape[1], train_X_gap.shape[2]),dtype='float32', name='gap_input')
print (gap_input)

Tensor("gap_input:0", shape=(?, 1, 1), dtype=float32)


In [10]:
lstm_out_stock = Embedding(output_dim=1, input_dim = 10000, input_length=1)(lstm_out_stock)

print (lstm_out_stock)

Tensor("embedding_1/Gather:0", shape=(?, 1, 1), dtype=float32)


In [11]:
merg_first = keras.layers.concatenate([lstm_out_stock, gap_input])
print (merg_first)
lstm_out_gap = LSTM(64)(merg_first)
lstm_out_gap = LSTM(64)(merg_first)
lstm_out_gap = Dense(1)(lstm_out_gap)
print (lstm_out_gap) 

Tensor("concatenate_1/concat:0", shape=(?, 1, 2), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 1), dtype=float32)


In [12]:
main_output = Dense(1, activation='linear', name='main_output')(lstm_out_gap)

In [15]:
model = Model(inputs=[stock_input, gap_input], outputs=[main_output])

In [22]:
model.compile(optimizer='rmsprop', loss='mse')
x={'stock_input':train_X_stock, 'gap_input': train_X_gap}
print (x)

{'gap_input': array([[[ 0.48803827]],

       [[ 0.43540668]],

       [[ 0.61722487]],

       [[ 0.40191385]],

       [[ 0.55980861]],

       [[ 0.39712918]],

       [[ 0.52631575]],

       [[ 0.41626793]],

       [[ 0.40191385]],

       [[ 0.39712918]],

       [[ 0.5119617 ]],

       [[ 0.5119617 ]],

       [[ 0.33492821]],

       [[ 0.54066986]],

       [[ 0.70334923]],

       [[ 0.71770334]],

       [[ 0.49282295]],

       [[ 0.45933014]],

       [[ 0.52153111]],

       [[ 0.3827751 ]],

       [[ 0.46889952]],

       [[ 0.46411481]],

       [[ 0.58851671]],

       [[ 0.55980861]],

       [[ 0.39234447]],

       [[ 0.44497606]],

       [[ 0.58851671]],

       [[ 0.63636363]],

       [[ 0.43062198]],

       [[ 0.47368419]],

       [[ 0.46889952]],

       [[ 0.42583731]],

       [[ 0.33971292]],

       [[ 0.42583731]],

       [[ 0.52153111]],

       [[ 0.31100479]],

       [[ 0.42583731]],

       [[ 0.49282295]],

       [[ 0.35885167]],

       [[ 0

In [21]:
history=model.fit(x={'stock_input':train_X_stock, 
                   'gap_input': train_X_gap
                   },
               y= {'main_output':train_y_stock,},
                 epochs=1, batch_size=1)

ValueError: Tried to convert 'x' to a tensor and failed. Error: None values not supported.